In [1]:
import numpy as np
from jinja2 import Environment, FileSystemLoader
from ase.atoms import Atoms
from ase.io import read
from ase.visualize import view

### Helper functions

In [2]:
def set_to_origin(arr):
    arr[:,0] -= arr[:,0].min()
    arr[:,1] -= arr[:,1].min()
    arr[:,2] -= arr[:,2].min()
    return arr

In [3]:
def center_by_index(arr, idx):
    arr[:,0] -= arr[idx,0]
    arr[:,1] -= arr[idx,1]
    arr[:,2] -= arr[idx,2]
    return arr

In [4]:
def move_to(arr, x, y, z):
    arr[:,0] += x
    arr[:,1] += y
    arr[:,2] += z
    return arr

In [5]:
def freeze_atoms(structure: Atoms, freeze_index = []) -> np.array:
    cons = np.ones((len(structure),3), dtype=int)
    for f in freeze_index:
        cons[f] = 0
    return cons

In [6]:
def pretty_coordinates(structure: Atoms, frozen_atoms = []):
    symbols = np.array(structure.get_chemical_symbols())
    symbols.shape = (len(symbols), 1) # Fix dimensions in order to concatenate
    matched = np.concatenate((symbols, structure.get_positions(), freeze_atoms(structure, frozen_atoms)), axis=1)
    pretty = ['      '.join(coord) for coord in matched]
    return pretty

In [7]:
def pretty_cell(structure: Atoms, distance: float):
    cell = structure.cell
    cell[2] = cell[2] + [0,0,distance]
    axis = [ c for c in cell]
    return [ '      '.join([str(i) for i in a]) for a in axis] # Doble recursion

### Main function

In [8]:
def build_mono_sub(z_distance, mono, sub, idx_sub):
    z_max = mono.positions[:,2].max() # uppermost postion of p layer
    y_max = mono.positions[:,1].max() 
    x_max = mono.positions[:,0].max() 
    y_min = mono.positions[:,1].min() 
    x_min = mono.positions[:,0].min() 
    sub.postions = center_by_index(sub.positions,idx_sub)
    delta_y = (y_max - y_min) / 2. + y_min
    delta_x = (x_max - x_min) / 2. + x_min
    sub.positions = move_to(sub.positions, delta_x, delta_y, z_max + z_distance)
    both_layers = mono + sub
    both_layers.cell[2,2] = both_layers.positions[:,2].max() + 5
    return both_layers

In [9]:
def gen_mono_sub_input(template_name: str,
    prefix: str, distance: float,
    mono, sub, idx_sub,
    frozen_atoms = []
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = build_mono_sub(distance, mono, sub, idx_sub)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "ntyp": len(set(both.get_chemical_symbols())),
    "coordenadas": pretty_coordinates(both, frozen_atoms),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

# Probando cosas

In [10]:
mono = read('mono4.cif')

In [11]:
mono22 = mono*(2,2,1)

In [15]:
acn = read('acn.cif')

In [16]:
view(build_mono_sub(2.0, mono22, acn, 2))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [17]:
gen_mono_sub_input('scf.in',
    'nitrilo', 2.33,
    mono22, acn, 2,
    []
)

In [19]:
[gen_mono_sub_input('scf.in', 'acn', i, mono22, acn, 2) for i in np.arange(2.00,5.5,0.5)]

[None, None, None, None, None, None, None]